In [ ]:
import numpy as np
import pandas as pd
from scipy.fft import fft, fftshift

# data = np.load('../all_data.npy)
antenna_locations = pd.read_csv("portmap.csv")
antenna_locations["comment"] = (
    antenna_locations["comment"].astype(str).str.replace("nan", "")
)
# antenna_locations = antenna_locations[
#     ~(
#         antenna_locations["comment"].str.contains("aux")
#         | antenna_locations["comment"].str.contains("unused")
#         | antenna_locations["comment"].str.contains("noise_ref")
#     )
# ]
antenna_locations["x_loc"] /= 100
antenna_locations["y_loc"] /= 100

## antenna WEMs 27 / 28 are not used
antenna_locations = antenna_locations[~antenna_locations["wem_no"].isin([27, 28])]

In [ ]:
# I think the unit on x_loc and y_loc is cm and the reference point is the center of the array.
# How to deal with dual polarization in practice? Two images? One image with both?


# The order of the channels in the np array is the index of the row - not the
# astro_chan number.
antenna_locations

In [ ]:
antenna_locations["r"] = np.sqrt(
    antenna_locations["x_loc"] ** 2 + antenna_locations["y_loc"] ** 2
)

The wave vector is given by:
$$ 
    \hat{w} = (\cos(\theta)\cos(\phi), \sin(\theta)\cos(\phi), \sin(\phi))
$$

where I'm assuming this is the Az / Alt relative to the way that the dish is currently pointing.

In [ ]:
THETA_RADIANS = 0
PHI_RADIANS = np.pi / 2

REFERENCE_WAVELENGTH = 1_480_000_000  # MHz

If this is pointing directly at VELA - then this should give the maximum of the peak signal when we beamform.

In [ ]:
def calc_phase_delay_radians(
    theta_radians: float,
    phi_radians: float,
    x_diff_m: float,
    y_diff_m: float,
    reference_wavelength: float,
) -> float:
    wave_vector = np.array(
        [
            np.cos(theta_radians) * np.cos(phi_radians),
            np.sin(theta_radians) * np.cos(phi_radians),
            np.sin(phi_radians),
        ]
    )

    # Does this need to be in u/v coordinates? theta would be measured from the x-axis and phi from the plane to the vector?
    # I don't think it matters so long as it's consistent (and we know the orientation of the PAF).
    baseline_m = np.array([x_diff_m, y_diff_m, 0])

    phase_delay_radians = (
        np.dot(wave_vector, baseline_m) * 2 * np.pi / reference_wavelength
    )

    return phase_delay_radians


def calc_phase_shift_for_antennae(
    theta_radians: float,
    phi_radians: float,
    antennae_map: pd.DataFrame,
    reference_wavelength: float,
) -> np.ndarray:
    output = np.zeros(len(antennae_map))
    for antenna in antennae_map.itertuples():
        output[antenna.Index] = np.exp(
            calc_phase_delay_radians(
                theta_radians,
                phi_radians,
                antenna.x_loc,
                antenna.y_loc,
                reference_wavelength,
            )
        )

    return output


def get_fold_segment(
    period_ms: float, seconds_between_samples: float, segment_size: int
) -> int:
    samples_in_period = period_ms / 1000 / seconds_between_samples

    fold_segment = int(round(samples_in_period / segment_size, 0))
    return fold_segment


def get_folded_data(
    data: np.ndarray,
    period_ms: float,
    seconds_between_samples: float,
    segment_size: int,
) -> np.ndarray:
    fold_segment = get_fold_segment(period_ms, seconds_between_samples, segment_size)

    output_folded = np.zeros((fold_segment, segment_size))

    num_folds = 0

    i = 0
    while True:
        start = i * fold_segment
        end = start + fold_segment
        if end > data.shape[0]:
            break
        output_folded += data[start:end]
        num_folds += 1
        i += 1
    output_folded /= num_folds

    return output_folded

In [ ]:
def get_fourier_power_spectrum(data: np.ndarray, segment_size: int) -> np.ndarray:
    window = np.hanning(segment_size)

    n_segments = (data.shape[1] - segment_size) // (segment_size)

    output = np.zeros((n_segments, segment_size))

    for j in range(n_segments):
        start = j * (segment_size)
        end = start + segment_size
        if end > data.shape[1]:
            print("past end of array")
            break
        sliced_data = data[0, start:end]

        ff_transform = fft(sliced_data * window)
        ff_shift = fftshift(ff_transform)
        power = np.abs(ff_shift)

        output[j, :] = power

    return output


def get_demeaned_scrunched_data(data: np.ndarray) -> np.ndarray:
    return data.mean(axis=1) - data.mean(axis=1).mean(axis=0, keepdims=True)

In [ ]:
data = np.load("../all_data.npy")

## Drop all the channels which are unused
a = (
    antenna_locations[antenna_locations["comment"] != ""]
    .reset_index()["index"]
    .tolist()
)

data = np.delete(data, list([int(i) for i in a]), axis=0)

antenna_locations = antenna_locations[antenna_locations["comment"] == ""].reset_index(
    drop=True
)

# data and antenna_locations should now be aligned.

In [ ]:
# Calc antenna phase shift
antenna_phase_shift = calc_phase_shift_for_antennae(
    THETA_RADIANS, PHI_RADIANS, antenna_locations, REFERENCE_WAVELENGTH
)
antenna_phase_shift = antenna_phase_shift[np.newaxis, :]


# Calculate weights
rs = antenna_locations["r"].to_numpy()

# trial this Gaussian weighting scheme - sigma chosen arbitrarily.
SIGMA = 1
weights = np.exp(-(rs**2) / (2 * SIGMA**2))
weights = weights[np.newaxis, :]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.fft import fftfreq

SEGMENT = 2**6
SAMPLING_RATE = int(51_200_000 / 27)  # Hz

OVERSAMPLED_BANDWIDTH = SAMPLING_RATE
BANDWIDTH = SAMPLING_RATE * 27 / 32
SKY_FREQUENCY = 926 * BANDWIDTH

In [ ]:
beam_formed_data = antenna_phase_shift @ data
weighted_and_phase_shifted = weights * antenna_phase_shift
weighted_beamformed_data = weighted_and_phase_shifted @ data

output = get_fourier_power_spectrum(beam_formed_data, SEGMENT)
output_weighted = get_fourier_power_spectrum(weighted_beamformed_data, SEGMENT)

VELA_PERIOD = 89.455


output_folded = get_folded_data(output, VELA_PERIOD, 27 / 51_200_000, SEGMENT)
output_weighted_folded = get_folded_data(
    output_weighted, VELA_PERIOD, 27 / 51_200_000, SEGMENT
)

FOLD_SEGMENT = get_fold_segment(VELA_PERIOD, 27 / 51_200_000, SEGMENT)

In [ ]:
n_segments = (beam_formed_data.shape[1] - SEGMENT) // (SEGMENT)


frequencies_axis = fftfreq(SEGMENT, 1 / SAMPLING_RATE)
frequencies_axis = np.fft.fftshift(frequencies_axis) + SKY_FREQUENCY
frequencies_axis_mhz = frequencies_axis / 1_000_000
times = np.arange(n_segments) * (SEGMENT) / int(SAMPLING_RATE)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 16), sharex=True, sharey=True)

ax = axes[0]
im = ax.plot(
    get_demeaned_scrunched_data(output),
)
ax.set_ylabel("Channel Frequency [Hz]")
ax.set_xlabel("Time [s]")


ax = axes[1]

im = ax.plot(
    get_demeaned_scrunched_data(output_weighted),
)
ax.set_ylabel("Channel Frequency [Hz]")
ax.set_xlabel("Time [s]")

plt.suptitle("Frequency vs Time for Multiple Channels", fontsize=16)

plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 8), sharex=True, sharey=True)

ax = axes[0]

im = ax.plot(
    times[:FOLD_SEGMENT],
    output_folded.mean(axis=1) - output_folded.mean(axis=1).mean(axis=0, keepdims=True),
)
ax.set_ylabel("Power (arb)")
ax.set_xlabel("Time [s]")


ax = axes[1]
im = ax.plot(
    times[:FOLD_SEGMENT],
    output_weighted_folded.mean(axis=1)
    - output_weighted_folded.mean(axis=1).mean(axis=0, keepdims=True),
)
ax.set_ylabel("Weighted Power (arb)")
ax.set_xlabel("Time [s]")


plt.suptitle("Power vs Time for Un-decayed Beamformed Data (Folded)", fontsize=16)

plt.show()

In [ ]:
## Look at getting peak signal-to-noise
from scipy.optimize import minimize


def max_func(x):
    return -dataset_to_optimize(x).max() * 1000


def dataset_to_optimize(x):
    theta_radians = THETA_RADIANS + x[0]
    phi_radians = PHI_RADIANS + x[1]

    antenna_phase_shift = calc_phase_shift_for_antennae(
        theta_radians, phi_radians, antenna_locations, REFERENCE_WAVELENGTH
    )
    antenna_phase_shift = antenna_phase_shift[np.newaxis, :]
    # Calculate weights
    rs = antenna_locations["r"].to_numpy()

    # trial this Gaussian weighting scheme - sigma chosen arbitrarily.
    SIGMA = 1 + x[2]
    weights = np.exp(-(rs**2) / (2 * SIGMA**2))
    weights = weights[np.newaxis, :]

    weighted_and_phase_shifted = weights * antenna_phase_shift
    weighted_beamformed_data = weighted_and_phase_shifted @ data
    output_weighted = get_fourier_power_spectrum(weighted_beamformed_data, SEGMENT)
    output_weighted_folded = get_folded_data(
        output_weighted, VELA_PERIOD, 27 / 51_200_000, SEGMENT
    )

    output_weighted_folded_power = get_demeaned_scrunched_data(output_weighted_folded)
    std_noise = output_weighted_folded_power[
        output_weighted_folded_power.shape[0] // 2 :
    ].std(axis=0)

    output_weighted_folded_power /= std_noise

    return output_weighted_folded_power


x = [0, 0, 0]
optimal = minimize(
    max_func, x, method="L-BFGS-B", bounds=[(None, None), (None, None), (-0.98, None)]
)
optimal

In [ ]:
optimal_data = dataset_to_optimize(optimal.x)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 8), sharex=True, sharey=True)


im = ax.plot(times[:FOLD_SEGMENT], optimal_data)
ax.set_ylabel("Power (arb)")
ax.set_xlabel("Time [s]")


plt.suptitle("Power vs Time for Un-decayed Beamformed Data (Folded)", fontsize=16)

plt.show()

In [ ]:
# Dedisperse data


VELA_DM = 67.99


def dedisperse_dataset(
    data: np.ndarray,
    dm: float,
    seconds_between_samples: float,
    frequencies_axis_mhz: np.ndarray,
    segment_size: int,
) -> np.ndarray:
    SECONDS_BETWEEN_SEGMENTS = seconds_between_samples * (segment_size)

    delay_samples = np.round(
        4.15
        * 10**3
        * dm
        * ((frequencies_axis_mhz**-2) - (frequencies_axis_mhz[-1] ** -2))
        / SECONDS_BETWEEN_SEGMENTS,
        0,
    ).astype(int)

    transposed_data = np.transpose(data, (1, 0)).copy()
    transposed_data_shape = transposed_data.shape

    final_data = np.zeros(
        shape=(
            transposed_data_shape[0],
            transposed_data_shape[1] - max(delay_samples),
        )
    )
    for i in range(delay_samples.shape[0]):
        if delay_samples[i] < max(delay_samples):
            final_data[i, :] = transposed_data[
                i, delay_samples[i] : (delay_samples[i] - max(delay_samples))
            ]
        else:
            final_data[i, :] = transposed_data[i, delay_samples[i] :]
    dedispersed_data = np.transpose(final_data, (1, 0))
    return dedispersed_data


dedispersed_folded_dataset = dedisperse_dataset(
    output_weighted_folded, VELA_DM, 27 / 51_200_000, frequencies_axis_mhz, SEGMENT
)

In [ ]:
def max_func(x):
    return -dataset_to_optimize(x).max() * 1000


def dataset_to_optimize(x):
    theta_radians = THETA_RADIANS + x[0]
    phi_radians = PHI_RADIANS + x[1]

    antenna_phase_shift = calc_phase_shift_for_antennae(
        theta_radians, phi_radians, antenna_locations, REFERENCE_WAVELENGTH
    )
    antenna_phase_shift = antenna_phase_shift[np.newaxis, :]
    # Calculate weights
    rs = antenna_locations["r"].to_numpy()

    # trial this Gaussian weighting scheme - sigma chosen arbitrarily.
    SIGMA = 1 + x[2]
    weights = np.exp(-(rs**2) / (2 * SIGMA**2))
    weights = weights[np.newaxis, :]

    weighted_and_phase_shifted = weights * antenna_phase_shift
    weighted_beamformed_data = weighted_and_phase_shifted @ data
    output_weighted = get_fourier_power_spectrum(weighted_beamformed_data, SEGMENT)
    output_weighted_folded = get_folded_data(
        output_weighted, VELA_PERIOD, 27 / 51_200_000, SEGMENT
    )
    output_weighted_folded_dedispersed = dedisperse_dataset(
        output_weighted_folded, VELA_DM, 27 / 51_200_000, frequencies_axis_mhz, SEGMENT
    )

    output_weighted_folded_power = get_demeaned_scrunched_data(
        output_weighted_folded_dedispersed
    )

    std_noise = output_weighted_folded_power[
        output_weighted_folded_power.shape[0] // 2 :
    ].std(axis=0)
    output_weighted_folded_power /= std_noise

    return output_weighted_folded_power


x = [0, 0, 0]
# L-BFGS-B respects bounds while Nelder-Mead does not
optimal = minimize(
    max_func, x, method="L-BFGS-B", bounds=[(None, None), (None, None), (-0.98, None)]
)
optimal

In [ ]:
optimal_data = dataset_to_optimize(optimal.x)

fig, ax = plt.subplots(1, 1, figsize=(10, 8), sharex=True, sharey=True)


im = ax.plot(times[: FOLD_SEGMENT - 10], optimal_data)
ax.set_ylabel("Power (arb)")
ax.set_xlabel("Time [s]")


plt.suptitle(
    "Power vs Time for Un-decayed Beamformed Data (Folded & De-dispersed)", fontsize=16
)

plt.show()